# Roctor Doster
---

This notebook aims to solve something of a personal issue of mine. My girlfriend works in the healthcare industry, so she is on call at the hospital four plus times a month. Part of this process is setting up a call roster which is a huge headache all around as the roster needs to give each doctor an almost equal number of calls whilst also catering to requests for calls on certain days. The model set out below will provide the optimum roster allocations whilst simultaneously meeting as many call requests as possible. This is but the first step in my master plan to put doctors out of work

In [ ]:
from IPython.display import Image
Image(url='https://media4.giphy.com/media/Rghq9s8RwVRyFxYYvB/giphy.gif')

In [ ]:
!pip install ortools

     |████████████████████████████████| 14.4MB 282kB/s 
     |████████████████████████████████| 1.0MB 28.1MB/s 
  Found existing installation: protobuf 3.12.4
    Uninstalling protobuf-3.12.4:
      Successfully uninstalled protobuf-3.12.4


In [ ]:
import datetime
import ipywidgets as widgets

Let's see if we can provide a UI for users to input their choices into

In [ ]:
dates = [datetime.date(datetime.datetime.now().year, i, 1) for i in range(1, 13)]
options = [(i.strftime('%b'), i) for i in dates]
widgets.SelectionRangeSlider(
    options=options,
    index=(0, 11),
    description='Months',
    disabled=False
)

SelectionRangeSlider(description='Months', index=(0, 11), options=(('Jan', datetime.date(2021, 1, 1)), ('Feb',…

In [ ]:
widgets.IntSlider(
    value=1,
    min=0,
    max=30,
    step=1,
    description='Num Doctors:',
    disabled=False,
    continuous_update=False,
    orientation='vertical',
    readout=True,
    readout_format='d'
)

IntSlider(value=1, continuous_update=False, description='Num Doctors:', max=30, orientation='vertical')

In [ ]:
from ortools.sat.python import cp_model


def main():
    # This script tries to find an optimal assignment of doctors to shifts
    # (2 shifts per day (as there needs to be two doctors on call on any day), for x days), subject to some constraints (see below).
    # Each doctor can request to be assigned on a specific day.
    # The optimal assignment maximizes the number of fulfilled shift requests.
    num_doctors = 3
    num_shifts = 2
    num_days = 7
    all_doctors = range(num_doctors)
    all_shifts = range(num_shifts)
    all_days = range(num_days)
    shift_requests = [[[1,1], [0,0], [0,0], [0,0], [1,0],
                       [1,0], [1,0]],
                      [[0,0], [0,0], [1,0], [1,0], [1,0],
                       [0,0], [1,0]],
                      [[1,0], [0,0], [0,0], [1,0], [0,0],
                       [1,0], [0,0]],
                      ]
    # Creates the model.
    model = cp_model.CpModel()

    #Output config info
    print(f'Number of doctors: {num_doctors}')
    print(f'Number of days in rotation: {num_days}\n')

    # Creates shift variables.
    # shifts[(n, d, s)]: nurse 'n' works shift 's' on day 'd'.
    shifts = {}
    for n in all_doctors:
        for d in all_days:
            for s in all_shifts:
                shifts[(n, d,
                        s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

    # Each shift is assigned to exactly one nurse in .
    for d in all_days:
        for s in all_shifts:
            model.Add(sum(shifts[(n, d, s)] for n in all_doctors) == 1)

    # Each doctor works at most one shift per day.
    for n in all_doctors:
        for d in all_days:
            model.Add(sum(shifts[(n, d, s)] for s in all_shifts) <= 1)

    # Try to distribute the shifts evenly, so that each doctor works
    # min_shifts_per_doctor shifts. If this is not possible, because the total
    # number of shifts is not divisible by the number of doctors, some doctors will
    # be assigned one more shift.
    min_shifts_per_doctor = (num_shifts * num_days) // num_doctors
    if num_shifts * num_days % num_doctors == 0:
        max_shifts_per_doctor = min_shifts_per_doctor
    else:
        max_shifts_per_doctor = min_shifts_per_doctor + 1
    for n in all_doctors:
        num_shifts_worked = 0
        for d in all_days:
            for s in all_shifts:
                num_shifts_worked += shifts[(n, d, s)]
        model.Add(min_shifts_per_doctor <= num_shifts_worked)
        model.Add(num_shifts_worked <= max_shifts_per_doctor)

    # pylint: disable=g-complex-comprehension
    model.Maximize(
        sum(shift_requests[n][d][s] * shifts[(n, d, s)] for n in all_doctors
            for d in all_days for s in all_shifts))
    # Creates the solver and solve.
    solver = cp_model.CpSolver()
    solver.Solve(model)
    for d in all_days:
        print('Day', d+1)
        for n in all_doctors:
            for s in all_shifts:
                if solver.Value(shifts[(n, d, s)]) == 1:
                    if shift_requests[n][d][s] == 1:
                        print('Doctor', n, 'works shift', s, '(requested).')
                    else:
                        print('Doctor', n, 'works shift', s, '(not requested).')
        print()

    # Statistics.
    print()
    print('Statistics')
    print('  - Number of shift requests met = %i' % solver.ObjectiveValue(),
          '(out of', num_doctors * min_shifts_per_doctor, ')')
    print('  - wall time       : %f s' % solver.WallTime())


if __name__ == '__main__':
    main()

Number of doctors: 3
Number of days in rotation: 7

Day 1
Doctor 0 works shift 1 (requested).
Doctor 2 works shift 0 (requested).

Day 2
Doctor 1 works shift 1 (not requested).
Doctor 2 works shift 0 (not requested).

Day 3
Doctor 0 works shift 1 (not requested).
Doctor 1 works shift 0 (requested).

Day 4
Doctor 0 works shift 1 (not requested).
Doctor 2 works shift 0 (requested).

Day 5
Doctor 1 works shift 0 (requested).
Doctor 2 works shift 1 (not requested).

Day 6
Doctor 0 works shift 0 (requested).
Doctor 1 works shift 1 (not requested).

Day 7
Doctor 0 works shift 0 (requested).
Doctor 1 works shift 1 (not requested).


Statistics
  - Number of shift requests met = 7 (out of 12 )
  - wall time       : 0.002659 s
